# Demo Notebook for "Automatic Differentiation With Higher Infinitesimals, or Computational Smooth Infinitesimal Analysis in Weil Algebra"

In this Notebook, we will demonstrate the functionality of Computational Smooth Infinitesimal Analysis.
Note that IHaskell emits superfluous like:

## Setup
This section prepares the environment for the computaion in what follows.
Genrally, you don't have to modify the contents of this section unless you know what you're doing.

NOTE: 

In [1]:
:set -Wno-all -Wno-name-shadowing -Wno-type-defaults

In [2]:
:set -XDataKinds -XPolyKinds -XTypeApplications -XTypeOperators -XGADTs -XFlexibleContexts -XMonoLocalBinds -XOverloadedStrings -XOverloadedLabels -XScopedTypeVariables -XFlexibleInstances -XMultiParamTypeClasses -XUndecidableInstances

In [3]:
import qualified Algebra.Prelude.Core as AP
import Numeric.Algebra.Smooth
import Numeric.Algebra.Smooth.Weil
import Algebra.Ring.Polynomial
import Algebra.Ring.Ideal
import IHaskell.Display
import IHaskell.Display.Blaze
import Control.Monad
import qualified Text.Blaze.Html5 as H5
import qualified Data.Map.Strict as M
import qualified Data.Text as T
import Data.Coerce
import GHC.OverloadedLabels
import Data.Reflection
import GHC.TypeLits
import qualified Data.Sized as SV
import Symbolic

type Q = AP.Rational
latexise = T.replace "*" "\\cdot" . T.replace "exp" "\\exp" . T.replace "cos" "\\cos" . T.replace "sin" "\\sin" . T.pack


instance
  ( c ~ Symbolic
  , Reifies i (WeilSettings n m)
  , KnownNat n
  , KnownNat m
  , KnownSymbol sym
  ) =>
  IsLabel sym (Weil i c)
  where
  fromLabel = injCoeWeil $ fromLabel @sym @Symbolic
  {-# INLINE fromLabel #-}


## Simple Univariate Differentiation

First, let's calculate the differential coefficients of 

$$
f(x) = \sin\left(\frac x 2\right) \mathrm{e}^{x^2}
$$

at $x = \frac \pi 4$ up to the fourth.

In [4]:
f :: Floating a => a -> a
f x = sin (x /2) * exp (x^2)

In [5]:
let dic = diffUpTo 4 f (pi/4)
M.toList dic

[(0,0.7091438342369428),(1,1.9699328611326816),(2,5.679986037666626),(3,19.85501973096302),(4,73.3133870997595)]

In [6]:
H5.table $ do
  H5.thead $ H5.tr $ do
    forM_ (M.keys dic) $ \i -> H5.th $ H5.toMarkup $ show i
  H5.tr $ do
    forM_ (M.toList dic) $ \(_, x) ->
      H5.td $ H5.toMarkup $ '$' : show x ++ "$"

0,1,2,3,4
$0.7091438342369428$,$1.9699328611326816$,$5.679986037666626$,$19.85501973096302$,$73.3133870997595$


As we saw in the paper (Theorem 2), we can compute $n$-th derivatives by evaluating $f$ in the tensor product of $\mathrm{R}[d]$'s, i.e. calculating $f(x + d_1 + \cdots + d_n)$ where $d_i \in \mathbb{R}[d_i]$.
Let's calculate up to the fourth:

In [7]:
let fval = f (pi/4 + di 0 + di 1 + di 2 + di 3) :: Weil (D1 |*| D1 |*| D1 |*| D1) Double
fval

73.3133870997595 d(0) d(1) d(2) d(3) + 19.855019730963026 d(0) d(1) d(2) + 19.855019730963026 d(0) d(1) d(3) + 19.855019730963026 d(0) d(2) d(3) + 19.855019730963026 d(1) d(2) d(3) + 5.679986037666626 d(0) d(1) + 5.679986037666626 d(0) d(2) + 5.679986037666626 d(1) d(2) + 5.679986037666626 d(0) d(3) + 5.679986037666626 d(1) d(3) + 5.679986037666626 d(2) d(3) + 1.9699328611326816 d(0) + 1.9699328611326816 d(1) + 1.9699328611326816 d(2) + 1.9699328611326816 d(3) + 0.7091438342369428

Here, the coefficient of $d_0 \cdots d_{n-1}$ corresponds to the $n$-th differential coefficient.

In [8]:
let pol = weilToPoly fval

H5.table $ do
  let ds :: [OrderedMonomial Grevlex 4]
      ds = map leadingMonomial (vars :: [Polynomial AP.Rational 4])
  H5.thead $ H5.tr $ do
    mapM_ (H5.th . H5.toMarkup . show) [0..4]
  H5.tr $ forM_ [0..4] $ \i ->
    H5.td $ do 
      "$"
      H5.toMarkup $ show $ AP.unwrapFractional $ coeff (AP.product $ take i ds) pol
      "$"

0,1,2,3,4
$ 0.7091438342369428 $,$ 1.9699328611326816 $,$ 5.679986037666626 $,$ 19.855019730963026 $,$ 73.3133870997595 $


However, this approach gives us exponential growth in $n$. 
As we saw in Lemma 2, we could use $\mathbb{R}[\varepsilon] = \mathbb{R}[x]/(x^{n+1})$ alternatively:

In [9]:
let fval' = f (pi/4 + di 0) :: Weil (DOrder 5) Double
fval'

3.054724462489979 d(0)^4 + 3.309169955160503 d(0)^3 + 2.839993018833313 d(0)^2 + 1.9699328611326816 d(0) + 0.7091438342369428

Note that the coefficien of $d^i$ is multiplied by $i!$, namely

$$
f(x + \varepsilon) = \sum_{0 \leq i \leq n} \frac{f^{(i)}}{i!}\varepsilon^i.
$$

Then the table gets:

In [10]:
higher = fmap AP.unwrapFractional $ terms $ weilToPoly fval'
higher

fromList [(1,0.7091438342369428),(X_0,1.9699328611326816),(X_0^2,2.839993018833313),(X_0^3,3.309169955160503),(X_0^4,3.054724462489979)]

In [11]:
H5.table $ do
  let dic = M.toList higher
  H5.thead $ H5.tr $ do
    H5.th ""
    mapM_ (H5.th . H5.toMarkup . show) [0..4]
  H5.tr $ do
    H5.th "$c_i$"
    forM_ dic $ \(_, c) -> H5.td $ "$" >> H5.toMarkup (show c) >> "$"
  H5.tr $ do
    H5.th "$c_i \\cdot i!$"
    forM_ (zip [0..] dic) $ \(i, (_,c)) -> H5.td $ "$" >> H5.toMarkup (show $ product [1..i] * c) >> "$"


,0,1,2,3,4
$c_i$,$ 0.7091438342369428 $,$ 1.9699328611326816 $,$ 2.839993018833313 $,$ 3.309169955160503 $,$ 3.054724462489979 $
$c_i \cdot i!$,$ 0.7091438342369428 $,$ 1.9699328611326816 $,$ 5.679986037666626 $,$ 19.85501973096302 $,$ 73.3133870997595 $


It coincides with the results we've gotten so far (modulo admissible floating-point errors)!

### Recovering symbolic differentiation
If we use `Symbolic` as a coefficient type instead of `Double`, we can recover the symbolic differentiation from the automatic differentiation!

In [12]:
let dic = normalise <$> f (#x + di 0) :: Weil (DOrder 3) Symbolic
H5.table $ do
  H5.thead $ H5.tr $ H5.th "n" *> H5.th "$f^{(n)}$"
  forM_ (M.toList $ terms $ weilToPoly dic) $ \(mon, p) -> H5.tr $  do
    H5.th $ H5.toMarkup $ show $ totalDegree mon
    H5.td $ do
      "$"
      H5.toMarkup $ latexise $ show $ AP.unwrapFractional p
      "$"

n,$f^{(n)}$
0,$ \sin (0.5 \cdot x) \cdot \exp (x \cdot x) $
1,$ \sin (0.5 \cdot x) \cdot \exp (x \cdot x) \cdot (x + x) + 0.5 \cdot \cos (0.5 \cdot x) \cdot \exp (x \cdot x) $
2,$ \sin (0.5 \cdot x) \cdot ((2.0 \cdot \exp (x \cdot x) + \exp (x \cdot x) \cdot (x + x) \cdot (x + x)) / 2.0) + 0.5 \cdot \cos (0.5 \cdot x) \cdot \exp (x \cdot x) \cdot (x + x) + 0.25 \cdot (- (\sin (0.5 \cdot x))) / 2.0 \cdot \exp (x \cdot x) $


## Multivariate differential

Let's see how the multivariate partial derivatives can be calculated with tensor products of Weil algebras.
Let 

$$
 g(x,y) = \sin(x) \mathrm{e}^{y^2}
$$

and calculate the partial derivatives at $(\frac{\pi}{3}, \frac{\pi}{6})$ of $g$ up to $(1,2)$-th.

In [13]:
g :: Floating x => x -> x -> x
g x y = sin x * exp (y ^ 2)

In [14]:
g' = g (pi/3 + di 0) (pi/6 + di 1) :: Weil (DOrder 2 |*| DOrder 3) Double
g'

1.0183395272163351 d(0) d(1)^2 + 0.6887520751685787 d(0) d(1) + 1.7638158004943616 d(1)^2 + 0.6577097839672799 d(0) + 1.1929535880104767 d(1) + 1.1391867624664787

In [15]:
let gDic :: M.Map (OrderedMonomial Grevlex 2) Double
    gDic = coerce $ terms $ weilToPoly g'

H5.table $ do
  forM_ (M.toList gDic) $ \(mon, coe) -> H5.tr $ do
    let np = product [1..totalDegree mon]
        degs = getMonomial mon
        xn = degs  SV.%!! 0
        yn = degs  SV.%!! 1
    H5.th $ do 
      "$"
      when (xn > 0) $ "\\partial x^{" >> H5.toMarkup (show xn) >> "}"
      when (yn > 0) $ "\\partial y^{" >> H5.toMarkup (show yn) >> "}"
      "g(x,y)"
      "$"
    H5.td $ "$" >> H5.toMarkup (show $ fromIntegral np * coe) >> "$"

"$ g(x,y) $",$ 1.1391867624664787 $
"$ \partial y^{ 1 } g(x,y) $",$ 1.1929535880104767 $
"$ \partial x^{ 1 } g(x,y) $",$ 0.6577097839672799 $
"$ \partial y^{ 2 } g(x,y) $",$ 3.527631600988723 $
"$ \partial x^{ 1 } \partial y^{ 1 } g(x,y) $",$ 1.3775041503371575 $
"$ \partial x^{ 1 } \partial y^{ 2 } g(x,y) $",$ 6.110037163298011 $


We can recover multivariate symbolic differentiation as well (this may take a while):

In [16]:
let g'' = normalise <$> g (#x + di 0) (#y + di 1) :: Weil (DOrder 2 |*| DOrder 3) Symbolic
    gDic :: M.Map (OrderedMonomial Grevlex 2) Symbolic
    gDic = coerce $ terms $ weilToPoly g''

H5.table $ do
  forM_ (M.toList gDic) $ \(mon, coe) -> H5.tr $ do
    let np = product [1..totalDegree mon]
        degs = getMonomial mon
        xn = degs  SV.%!! 0
        yn = degs  SV.%!! 1
    H5.th $ do 
      "$"
      when (xn > 0) $ "\\partial x^{" >> H5.toMarkup (show xn) >> "}"
      when (yn > 0) $ "\\partial y^{" >> H5.toMarkup (show yn) >> "}"
      "g(x,y)"
      "$"
    H5.td $ "$" >> H5.toMarkup (latexise $ show $ fromIntegral np * coe) >> "$"

"$ g(x,y) $",$ 1.0 \cdot (\sin x \cdot \exp (y \cdot y)) $
"$ \partial y^{ 1 } g(x,y) $",$ 1.0 \cdot (\sin x \cdot \exp (y \cdot y) \cdot (y + y)) $
"$ \partial x^{ 1 } g(x,y) $",$ 1.0 \cdot (\cos x \cdot \exp (y \cdot y)) $
"$ \partial y^{ 2 } g(x,y) $",$ 2.0 \cdot (\sin x \cdot ((2.0 \cdot \exp (y \cdot y) + \exp (y \cdot y) \cdot (y + y) \cdot (y + y)) / 2.0)) $
"$ \partial x^{ 1 } \partial y^{ 1 } g(x,y) $",$ 2.0 \cdot (\cos x \cdot \exp (y \cdot y) \cdot (y + y)) $
"$ \partial x^{ 1 } \partial y^{ 2 } g(x,y) $",$ 6.0 \cdot (\cos x \cdot ((2.0 \cdot \exp (y \cdot y) + \exp (y \cdot y) \cdot (y + y) \cdot (y + y)) / 2.0)) $


## Computation in General Weil Algebra

We can treat general Weil algebra defined by general ideal over polynomial rings.
Let us consider the following randomly-chosen ideal:

$$
I = \left\langle x^3 - 2 * y^2, x ^ 2 y, y ^ 3 \right\rangle.
$$

Let's test if $I$ is Weil or not:

In [17]:
i :: Ideal (Polynomial AP.Rational 2)
i = toIdeal [var 0 ^ 3 - 2 * var 1 ^2 , var 0 ^2 * var 1, var 1 ^ 3]

isWeil i

Just (SomeWeil (WeilSettings {weilBasis = [[0,0],[0,1],[1,0],[0,2],[1,1],[2,0],[1,2]], nonZeroVarMaxPowers = [4,2], weilMonomDic = fromList [([0,2],[0,0,0,1,0,0,0]),([1,1],[0,0,0,0,1,0,0]),([3,0],[0,0,0,2,0,0,0]),([1,0],[0,0,1,0,0,0,0]),([4,0],[0,0,0,0,0,0,2]),([2,0],[0,0,0,0,0,1,0]),([1,2],[0,0,0,0,0,0,1]),([0,1],[0,1,0,0,0,0,0]),([0,0],[1,0,0,0,0,0,0])], table = fromList [((0,0),1),((0,1),X_1),((1,2),X_0*X_1),((0,2),X_0),((1,1),X_1^2),((0,3),X_1^2),((2,5),2*X_1^2),((0,4),X_0*X_1),((2,2),X_0^2),((0,5),X_0^2),((2,3),X_0*X_1^2),((0,6),X_0*X_1^2),((1,4),X_0*X_1^2),((5,5),2*X_0*X_1^2)]}))

OK, it is a Weil algebra (although its intution is unclear). Let's evaluate some function on it:

In [18]:
import Data.Maybe

fromJust $ withWeil i $ 
  let x = pi / 6 + di 0
      y = pi / 3 + di 1
  in x * y * sin x * exp (y*y)

14.68798022716167*X_0*X_1^2 + 2.304875237017878*X_0^2 + 9.115643688794316*X_0*X_1 + 2.2212016147625855*X_1^2 + 2.9893974579377462*X_0 + 2.502984251874212*X_1 + 0.8208322983278699

It somehow works indeed!

The algorithm rejects non-Weil algebra correctly.
First let us consider $\langle x^2 - 1 \rangle \subseteq \mathbb{R}[x]$, which is zero-dimensional but not nilpotent:

In [19]:
withWeil (toIdeal [var 0 ^2 - 1 :: Polynomial Q 1]) $ 
  let x = pi / 6 + di 0
      y = pi / 3 + di 1
  in x * y * sin x * exp (y*y)

Nothing

It returned `Nothing`, which means it is not Weil.

Next we consider The  $\langle x^2 - y^3 \rangle \subseteq \mathbb{R}[x, y]$.
Note that $y$ never vanish so it is not even zero-dimensional!

In [20]:
withWeil (toIdeal [var 0 ^2 - var 1 ^ 3 :: Polynomial Q 2]) $ 
  let x = pi / 6 + di 0
      y = pi / 3 + di 1
  in x * y * sin x * exp (y*y)

Nothing

It returns `Nothing` as expected.